In [1]:
 categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, 
                                  random_state=42)

In [3]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [4]:
len(twenty_train.data)

2257

In [5]:
len(twenty_train.filenames)

2257

In [6]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [7]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [8]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int32)

In [10]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [12]:
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [14]:
X_train_counts.shape

(2257, 35788)

In [15]:
count_vect.vocabulary_.get(u'algorithm')

4690

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer

In [18]:
tf_transformer = TfidfTransformer(use_idf = False).fit(X_train_counts)

In [19]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [20]:
X_train_tf.shape

(2257, 35788)

In [21]:
tfidf_transformer = TfidfTransformer()

In [22]:
X_train_tfidf =tfidf_transformer.fit_transform(X_train_counts)

In [23]:
X_train_tfidf.shape

(2257, 35788)

In [26]:
## Training a classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [27]:
docs_new = ["God is love", "OpenGL on the GPU is fast"]

In [29]:
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [30]:
predicted = clf.predict(X_new_tfidf)

In [31]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


In [ ]:
## Building a pipeline

In [33]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB())])

In [34]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [37]:
## Evaluation of the performance on the test set
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
                                categories = categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.83488681757656458

In [38]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty = 'l2',
                                         alpha = 1e-3, random_state=42,
                                         max_iter=5, tol=None))])

In [39]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [41]:
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9127829560585885

In [42]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,
                                   target_names = twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [43]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)

In [44]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range' : [(1, 1), (1, 2)],
             'tfidf__use_idf' : (True, False),
             'clf__alpha' : (1e-2, 1e-3)}

In [45]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

In [47]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [48]:
twenty_train.target_names[gs_clf.predict(["God i slove"])[0]]

'soc.religion.christian'

In [49]:
gs_clf.best_score_

0.90000000000000002

In [50]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" %(param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
